<a href="https://colab.research.google.com/github/MartinMashalov/ArbitrageBetting/blob/main/SureBetsBasketball.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [156]:
import http.client
import json 
from requests import get
from datetime import date
import pandas as pd
from pprint import pprint 
import numpy as np
from tqdm import tqdm 

In [157]:
api_key: str = 'd05957b6db564a11a84af741f5c84ac7'
conn = http.client.HTTPSConnection("v1.basketball.api-sports.io")
headers_conn = {
    'x-rapidapi-host': "v1.basketball.api-sports.io",
    'x-rapidapi-key': api_key 
    }
headers_req: dict = {'x-apisports-key': api_key}

In [158]:
# get possible bookmakers
conn.request("GET", "/bookmakers", headers=headers_conn)
res = conn.getresponse()
data = res.read().decode("utf-8")
data = json.loads(data)['response']
bookmakers: dict = {i['id']:i['name'] for i in data if i['name']}

In [159]:
today = str(date.today())
url: str = f"https://v1.basketball.api-sports.io/games?date={str(date.today())}"
# get all the games coming up on a specific date
games = get(url, headers=headers_req).json()['response']
# filter for all the NBA games
games = [i for i in games if i['country']['name'] == 'USA' or i['country']['name'] == 'Europe']
games = [{'game_id': i['id'], 'home': i['teams']['home']['name'], 'home': i['teams']['away']['name'], 'league': i['league']['name']} for i in games]
df = pd.DataFrame.from_dict(games)

In [160]:
game_odds = []
game_names = df['home'].to_list()
game_leagues = df['league'].to_list()
for game_id in tqdm(df['game_id'].unique()): 
  url: str = f"https://v1.basketball.api-sports.io/odds?game={str(game_id)}"
  try: 
    odds_data = get(url, headers=headers_req).json()['response'][0]['bookmakers']
  except (IndexError, KeyError): 
    continue
  game_odds.append(odds_data)

100%|██████████| 22/22 [00:08<00:00,  2.70it/s]


In [164]:
import itertools
from itertools import combinations
import ast

# create containers for data storage
meta_data = {} # collection of dataframes by bet types by sportsbook
processed_bet_types: list = []
odds_matches: dict = {}

def create_key_name(bet_name: str, bet_values_headers: list): 
  """create the name of a key for the bet name"""
  final_bet_name: str = f'{bet_name} - '
  for idx, bet_value in enumerate(bet_values_headers): 
    if idx != len(bet_values_headers) - 1: 
      final_bet_name += f'{bet_value} - '
    else: 
      final_bet_name += f'{bet_value}'

def hash_map_restructure(hash_map: dict): 
  """restructure the hash map to make nested computation more efficient"""
  new_map: dict = {}
  try: 
    for nested_bet in hash_map['bets']: 
      if not any(char.isdigit() for char in nested_bet['values'][0]['value']): 
        continue
      for value in nested_bet['values']: 
        new_map[f"{nested_bet['name']} - {value['value']}"] = value['odd']
    hash_map['bets'] = new_map
    return hash_map
  except (TypeError, KeyError): 
    return hash_map

def check_bet_availability(name: str, bet_arr: list) -> bool:
  """check if the bet is available in a certain sportbook"""
  for bet in bet_arr: 
    if bet['name'] == name: 
      return True
  return False

def plus_ev(matrix: np.array):
  """positive ev checker"""
  # find the sum of the inverted diagonals
  diag_normal_sum = np.sum(np.diagonal(matrix))
  matrix[:, [-1, 0]] = matrix[:, [0, -1]]
  diag_inverse_sum = np.sum(np.diagonal(matrix))

  # check if the sums satisfy positive ev condition
  if diag_normal_sum < 1: 
    return 1
  elif diag_inverse_sum < 1: 
    return 2
  else: 
    return 3
  
def check_arbitrage_pairs(firstbook, secondbook, bet_type, game, league, *book_values):
  """check if an arbitrage oppurtunity exists between books"""
  odds_combo_arr: dict = {}
  odds_combo_filtered: dict = {}

  # gather all the odd values
  for book_value_a in book_values[0]['values']: 
    if book_value_a['value'] in [i['value'] for i in book_values[1]['values']]: 
      odds_combo_arr[book_value_a['value']] = [float(book_value_a['odd']), float([i['odd'] for i in book_values[1]['values'] if i['value'] == book_value_a['value']][0])]
  
  seen_opp_keys: list = []
  for key, item in odds_combo_arr.items():
    # find the opposite key
    if 'Handicap' in bet_type: 
      if 'Home' in key: 
        opp_key = key.replace('Home', 'Away')
      else: 
        opp_key = key.replace('Away', 'Home')
    elif 'Over/Under' in bet_type: 
      if 'Over' in key: 
        opp_key = key.replace('Over', 'Under')
      else: 
        opp_key = key.replace('Under', 'Over')
    else: 
      if 'Yes' in key: 
        opp_key = key.replace('Yes', 'No')
      else: 
        opp_key = key.replace('No', 'Yes')

    # make selection matrix
    if opp_key not in seen_opp_keys: 
      try: 
        arr = np.array([[1/i for i in item], [1/i for i in odds_combo_arr[opp_key]]])
      except KeyError: 
        continue
      seen_opp_keys.append(opp_key)
    
      # create the matrix
      matrix = np.array(arr)
      ev_checker = plus_ev(matrix)

      # check for the +ev bets
      if ev_checker == 1: 
        odds_combo_filtered[f'{game} - {league} - {firstbook}:{secondbook} + {key} - {opp_key}'] = [item[0], odds_combo_arr[opp_key][1]]
      elif ev_checker == 2: 
        odds_combo_filtered[f'{game} - {league} - {firstbook}:{secondbook} + {key} - {opp_key}'] = [item[1], odds_combo_arr[opp_key][0]]
      else: 
        continue

  return odds_combo_filtered

def run(): 
  """main run function"""
  arb_collection = []

  for game_id_searcher, idx in enumerate(range(len(game_odds))): 
    data = game_odds[idx]
    # create all possible pairs of books
    book_pairs = list(combinations(data, 2))
    book_triplets = list(combinations(data, 3))

    # loop through each of the pairs to find arbitrage pairs
    for sportsbook_pair in tqdm(book_pairs): 
      bets = sportsbook_pair[0]['bets']
      for bet in bets: 
        if check_bet_availability(bet['name'], sportsbook_pair[1]['bets']): 
          if 'Over/Under' in bet['name'] or 'Odds/Even' in bet['name'] or 'Overtime' in bet['name'] or 'Both' in bet['name']:  #'Handicap' in bet['name']
            comparison_bet = [i for i in sportsbook_pair[1]['bets'] if i['name'] == bet['name']][0]
            arbitrage_data = check_arbitrage_pairs(sportsbook_pair[0]['name'], sportsbook_pair[1]['name'], bet['name'], game_names[game_id_searcher], game_leagues[game_id_searcher], bet, comparison_bet)
            if arbitrage_data: 
              arb_collection.append(arbitrage_data)
  
  # unpack the arbitrage container with all the bets
  final_container: dict = []
  for parcel in arb_collection: 
    for item, key in parcel.items(): 
      final_container.append([item, *key])
  return final_container

container = run()
df = pd.DataFrame(container, columns=['BetName', 'OddsA', 'OddsB'])  
df['Profitability'] = 1- 1/df['OddsA'] - 1/df['OddsB']
df = df[df['Profitability'] > 0.05]
df.sort_values(by='Profitability', inplace=True, ascending=False)

100%|██████████| 153/153 [00:00<00:00, 2189.07it/s]


In [165]:
def compute_weights(odds_a, odds_b, position_size, max_iter: int = 10000): 
  """find the optimal bet sizing"""
  # set the numpy random seed
  np.random.seed(42)

  # create sampling function for position sizing fractions with random seed set from before
  rng = np.random.default_rng(seed=42)

  # select position sizing at percentage level for both sides of the trade
  sample = rng.integers(20, 80, size=max_iter) / 100
  inverse_sample = 1 - sample

  # compute expected value of each position by hadamard product
  sample_filter_a = np.multiply(position_size*sample, np.full((1, len(sample)), odds_a)[0])
  sample_filter_b = np.multiply(position_size*inverse_sample, np.full((1, len(inverse_sample)), odds_b)[0])
  # find the valid position sizes that satisfy arbitrage
  valid_args_a = np.argwhere(sample_filter_a > position_size)
  valid_args_b = np.argwhere(sample_filter_b > position_size)
  valid_args = np.intersect1d(valid_args_a, valid_args_b)

  # filter and create the opposite position sizes
  sample, inverse_sample = sample[valid_args], inverse_sample[valid_args]
  
  # create the sizing matrix
  sizing_matrix = np.transpose(np.array([sample, inverse_sample]))
  sizing_matrix *= position_size
  odds_matrix = np.array([odds_a, odds_b])

  # multiple the odds matrix and position sizing matrix
  result_matrix = np.matmul(sizing_matrix, odds_matrix)
  
  # find the maximum value index of the resulting matrix
  efficient_idx = np.argmax(result_matrix)
  return round(position_size*sample[efficient_idx], 2), round(position_size*inverse_sample[efficient_idx], 2)
  
position_size = 100
df['SportsbookA'] = df['BetName'].apply(lambda x: str(x).split(':')[0].split('- ')[-1])
df['SportsbookB'] = df['BetName'].apply(lambda x: str(x).split(':')[1].split(' Under')[0].split(' Over')[0].split('- ')[-1].replace(' +', ''))
df['Game'] = df['BetName'].apply(lambda x: ' '.join(str(x).split("-")[0:2]))
df['Bet'] = df['BetName'].apply(lambda x: ''.join(''.join(str(x).split('-')[-2:]).split('+')[1:]))
df['BetInfo'] = df.apply(lambda x: compute_weights(x.OddsA, x.OddsB, position_size), axis=1)
df.reset_index(inplace=True, drop=True)
df = df[['OddsA', 'OddsB', 'Profitability', 'Bet', 'SportsbookA', 'SportsbookB', 'Game', 'BetInfo']]
df.head(50)

,OddsA,OddsB,Profitability,Bet,SportsbookA,SportsbookB,Game,BetInfo
0,1.87,2.60,0.080625,Over 224 Under 224,NordicBet,10Bet,Cibona ABA League,"(54.0, 46.0)"
1,2.60,1.87,0.080625,Under 224 Over 224,Betsson,10Bet,Cibona ABA League,"(46.0, 54.0)"
2,1.87,2.60,0.080625,Over 224 Under 224,Betsson,10Bet,Cibona ABA League,"(54.0, 46.0)"
3,2.60,1.87,0.080625,Under 224 Over 224,NordicBet,10Bet,Cibona ABA League,"(46.0, 54.0)"
4,2.50,1.91,0.076440,Under 224.5 Over 224.5,10Bet,WilliamHill,Cibona ABA League,"(47.0, 53.0)"
5,1.91,2.50,0.076440,Over 224.5 Under 224.5,10Bet,WilliamHill,Cibona ABA League,"(53.0, 47.0)"
6,2.55,1.87,0.073084,Under 224 Over 224,Betsson,ComeOn,Cibona ABA League,"(46.0, 54.0)"
7,1.87,2.55,0.073084,Over 224 Under 224,Betsson,ComeOn,Cibona ABA League,"(54.0, 46.0)"
8,1.87,2.55,0.073084,Over 224 Under 224,NordicBet,ComeOn,Cibona ABA League,"(54.0, 46.0)"
9,2.55,1.87,0.073084,Under 224 Over 224,NordicBet,ComeOn,Cibona ABA League,"(46.0, 54.0)"
